* # Imports ( please turn on internet for the imports and configs parts, there is no unwanted codes. Please go through the notebook for verification ) 

In [ ]:
%%capture
!pip install pyctcdecode
!python -m pip install pypi-kenlm
!pip install jiwer
!pip install bnunicodenormalizer
!pip install datasets
!pip install transformers

In [ ]:
%%capture
!pip install datasets
!pip install transformers
# !pip install torchaudio
!pip install huggingsound
!pip -q install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode

In [ ]:
import os
import numpy as np
from tqdm.auto import tqdm
from glob import glob
from transformers import AutoFeatureExtractor, pipeline
import pandas as pd
import librosa
import IPython
from datasets import load_metric
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import re
import gc
import wave
from scipy.io import wavfile
import scipy.signal as sps

# import torchaudio
from IPython.display import Audio, display
# from aksharamukha import transliterate
import random

from bnunicodenormalizer import Normalizer 

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True,nb_workers=8)


# print(torch.__version__)
# print(torchaudio.__version__)

bnorm=Normalizer()
import re
import pandas as pd
from pyctcdecode import build_ctcdecoder
from transformers import Wav2Vec2ProcessorWithLM
from transformers import AutoModelForCTC, Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC

# Configs

In [ ]:
from transformers import (
                          CONFIG_MAPPING,
                          MODEL_FOR_MASKED_LM_MAPPING,
                          MODEL_FOR_CAUSAL_LM_MAPPING,
                          PreTrainedTokenizer,
                          TrainingArguments,
                          AutoConfig,
                          AutoTokenizer,
                          AutoModelWithLMHead,
                          AutoModelForCausalLM,
                          AutoModelForMaskedLM,
                          LineByLineTextDataset,
                          TextDataset,
                          DataCollatorForLanguageModeling,
                          DataCollatorForWholeWordMask,
                          DataCollatorForPermutationLanguageModeling,
                          PretrainedConfig,
                          Trainer,
                          set_seed,
                          )

In [ ]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\।\’]'

train_df = pd.read_csv('../input/dlsprint/train.csv')
valid_df = pd.read_csv('../input/dlsprint/validation.csv')
with open('text.txt', 'w') as f:
    for sentence in train_df['sentence']:
        f.write(re.sub(chars_to_ignore_regex, '', sentence))
        f.write(' ')
        
    for sentence in valid_df['sentence']:
        f.write(re.sub(chars_to_ignore_regex, '', sentence))
        f.write(' ')

In [ ]:
! sudo apt -y install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev
! wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
! mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
! ls kenlm/build/bin
! kenlm/build/bin/lmplz -o 6 < "text.txt" > "5gram.arpa"

In [ ]:
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

In [ ]:
import kenlm
model = kenlm.LanguageModel('./5gram_correct.arpa')

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("../input/sammodel1/checkpoint-900")
# vocab_dict = processor.tokenizer.get_vocab()
vocab_dict = {'<pad>': 0,
 '<s>': 1,
 '</s>': 2,
 '<unk>': 3,
 'ই': 4,
 '3': 5,
 'হ': 6,
 '…': 7,
 'ল': 8,
 '্': 9,
 'ৈ': 10,
 'ো': 11,
 '৪': 12,
 'ধ': 13,
 'উ': 14,
 'া': 15,
 'ঞ': 16,
 'F': 17,
 'অ': 18,
 'ও': 19,
 'ট': 20,
 'খ': 21,
 'ড়': 22,
 'স': 23,
 '০': 24,
 'ম': 25,
 'ং': 26,
 'ৌ': 27,
 'গ': 28,
 'ঃ': 29,
 '\u200c': 30,
 'থ': 31,
 'e': 32,
 'ি': 33,
 'ষ': 34,
 '৯': 35,
 '়': 36,
 'চ': 37,
 'শ': 38,
 'ৗ': 39,
 'ঊ': 40,
 '৬': 41,
 'ঈ': 42,
 'ঋ': 43,
 'ঠ': 44,
 'ত': 45,
 'এ': 46,
 '৫': 47,
 'আ': 48,
 'ছ': 49,
 'ূ': 50,
 'ব': 51,
 'ঐ': 52,
 'প': 53,
 'ী': 54,
 'ড': 55,
 '৭': 56,
 'ণ': 57,
 'ফ': 58,
 'ু': 59,
 'ৃ': 60,
 '১': 61,
 '|': 62,
 '৮': 63,
 '\u200d': 64,
 'i': 65,
 'ৰ': 66,
 'ঔ': 67,
 'ভ': 68,
 '\u200e': 69,
 'ঙ': 70,
 'ৎ': 71,
 'ঘ': 72,
 'দ': 73,
 '২': 74,
 'ঝ': 75,
 'l': 76,
 'য়': 77,
 'জ': 78,
 'ক': 79,
 'ন': 80,
 'য': 81,
 'ে': 82,
 'র': 83,
 '৩': 84,
 'ঢ': 85,
 'ঁ': 86}
# sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}
sorted_vocab_dict = vocab_dict

In [ ]:
decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

In [ ]:


processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import gc
# del variables
gc.collect()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
asr = pipeline("automatic-speech-recognition", model="../input/sammodel1/checkpoint-900",tokenizer=processor_with_lm.tokenizer, feature_extractor=processor_with_lm.feature_extractor, decoder=processor_with_lm.decoder,device=0)



In [ ]:
class bn_asr_Dataset(Dataset):
    '''
    args:
        df      : path of the dataframe
        dir     : directory of sound files
    '''
    def __init__(self,df,dir):
        self.df = pd.read_csv(df)
        self.dir = dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
   
        #speech, _ = librosa.load(self.dir+self.df.path[i], sr=feature_extractor.sampling_rate) 
        path = self.dir+self.df.path[i]
        path = os.path.splitext(path)[0]+'.wav'
        # Read file
        sampling_rate, data = wavfile.read(path)
        # Resample data
        number_of_samples = round(len(data) * float(processor_with_lm.feature_extractor.sampling_rate) / sampling_rate)
        speech = sps.resample(data, number_of_samples)
        return speech
  


In [ ]:

def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None]) 


In [ ]:
df = pd.read_csv('../input/dlsprint/sample_submission.csv')
len(df.path)

*  Directory infer Function

In [ ]:
def directory_infer(test_dataset):
    class CFG:
        model_name = processor_with_lm 
        valid_df_path = '../input/traincsv/mycsvfile.csv'
        sample_sub_df_path = '../input/dlsprint/sample_submission.csv'
        valid = "../input/dlsprint/train_files/"
        test = '../input/dlsprint/test_files/'
        valid_wav = '../input/validation-fileswav-format/validation_files_wav/'
        test_wav = test_dataset
        batch_size = 48#not using this param now
        single_SPEECH_FILE = "../input/dlsprint/train_files/common_voice_bn_30614352.mp3"
        post_asr_corrector = True
    
    test_dataset = bn_asr_Dataset(CFG.sample_sub_df_path,CFG.test_wav)
    for i in range(len(test_dataset)):
        pred = asr(test_dataset.__getitem__(i), chunk_length_s=112, stride_length_s=None)
    
    #applying simple post processing with error handler
        try:
            if (pred["text"][-1] == '।'):
                df.sentence[i] = pred["text"]
                df.sentence[i]=df.sentence[i].replace("<>"," ")
                df.sentence[i]=df.sentence[i].replace("  "," ")
                df.sentence[i]=df.sentence[i].replace("<unk>"," ")
                df.sentence[i]=df.sentence[i].replace("<s>"," ")
                df.sentence[i]=df.sentence[i].replace("</s>"," ")
                df.sentence[i]=df.sentence[i].replace("f"," ")
                df.sentence[i]=df.sentence[i].replace("F"," ")
                df.sentence[i]=df.sentence[i].replace("\u200c"," ")
                df.sentence[i]=df.sentence[i].replace("\u200d"," ")
                df.sentence[i]=df.sentence[i].replace("\u200e"," ")
                df.sentence[i]=df.sentence[i].replace("…"," ")
            else:
                df.sentence[i] = pred["text"]+'।'
                df.sentence[i]=df.sentence[i].replace("<>"," ")
                df.sentence[i]=df.sentence[i].replace("  "," ")
                df.sentence[i]=df.sentence[i].replace("<unk>"," ")
                df.sentence[i]=df.sentence[i].replace("<s>"," ")
                df.sentence[i]=df.sentence[i].replace("</s>"," ")
                df.sentence[i]=df.sentence[i].replace("f"," ")
                df.sentence[i]=df.sentence[i].replace("F"," ")
                df.sentence[i]=df.sentence[i].replace("\u200c"," ")
                df.sentence[i]=df.sentence[i].replace("\u200d"," ")
                df.sentence[i]=df.sentence[i].replace("\u200e"," ")
                df.sentence[i]=df.sentence[i].replace("…"," ")

        except:
            df.sentence[i] = pred["text"]+'।'
            df.sentence[i]=df.sentence[i].replace("<>"," ")
            df.sentence[i]=df.sentence[i].replace("  "," ")
            df.sentence[i]=df.sentence[i].replace("<unk>"," ")
            df.sentence[i]=df.sentence[i].replace("<s>"," ")
            df.sentence[i]=df.sentence[i].replace("</s>"," ")
            df.sentence[i]=df.sentence[i].replace("f"," ")
            df.sentence[i]=df.sentence[i].replace("F"," ")
            df.sentence[i]=df.sentence[i].replace("\u200c"," ")
            df.sentence[i]=df.sentence[i].replace("\u200d"," ")
            df.sentence[i]=df.sentence[i].replace("\u200e"," ")
            df.sentence[i]=df.sentence[i].replace("…"," ")
        
    df.sentence=df.sentence.parallel_apply(lambda x:normalize(x)) #unicode normalizer
    return df



For single wav file of 16000hz

In [ ]:

def infer(audio_path):
    speech, sr = librosa.load(audio_path, sr=16000)
    prediction = asr(speech, chunk_length_s=112, stride_length_s=None)
    pred = prediction["text"]
    if (pred[-1] == '।'):
        pred = pred
        pred = pred.replace("<>"," ")
        pred = pred.replace("  "," ")
        pred = pred.replace("<unk>"," ")
        pred = pred.replace("<s>"," ")
        pred = pred.replace("</s>"," ")
        pred = pred.replace("<f>"," ")
        pred = pred.replace("<F>"," ")
        pred = pred.replace("\u200c"," ")
        pred = pred.replace("\u200d"," ")
        pred = pred.replace("\u200e"," ")
        pred = pred.replace("…"," ")

    else:
        pred = pred+'।'
        pred = pred.replace("<>"," ")
        pred = pred.replace("  "," ")
        pred = pred.replace("<unk>"," ")
        pred = pred.replace("<s>"," ")
        pred = pred.replace("</s>"," ")
        pred = pred.replace("<f>"," ")
        pred = pred.replace("<F>"," ")
        pred = pred.replace("\u200c"," ")
        pred = pred.replace("\u200d"," ")
        pred = pred.replace("\u200e"," ")
        pred = pred.replace("…"," ")
    pred=normalize(pred)
    return pred


    

In [ ]:
def batch_infer(audio_paths, batch_size):
    test_dataset=audio_paths
    class CFG:
        model_name = processor_with_lm 
        valid_df_path = '../input/traincsv/mycsvfile.csv'
        sample_sub_df_path = '../input/dlsprint/sample_submission.csv'
        valid = "../input/dlsprint/train_files/"
        test = '../input/dlsprint/test_files/'
        valid_wav = '../input/faster-way-to-convert-mp3-to-wav-using-joblib/'
        test_wav = test_dataset
        batch_size = 48#not using this param now
        single_SPEECH_FILE = "../input/dlsprint/train_files/common_voice_bn_30614352.mp3"
        post_asr_corrector = True
    
    test_dataset = bn_asr_Dataset(CFG.sample_sub_df_path,CFG.test_wav)
    for i in range(len(test_dataset)):
        pred = asr(test_dataset.__getitem__(i), chunk_length_s=112, stride_length_s=None)
    
    #applying simple post processing with error handler
        try:
            if (pred["text"][-1] == '।'):
                df.sentence[i] = pred["text"]
                df.sentence[i]=df.sentence[i].replace("<>"," ")
                df.sentence[i]=df.sentence[i].replace("  "," ")
                df.sentence[i]=df.sentence[i].replace("<unk>"," ")
                df.sentence[i]=df.sentence[i].replace("<s>"," ")
                df.sentence[i]=df.sentence[i].replace("</s>"," ")
                df.sentence[i]=df.sentence[i].replace("f"," ")
                df.sentence[i]=df.sentence[i].replace("F"," ")
                df.sentence[i]=df.sentence[i].replace("\u200c"," ")
                df.sentence[i]=df.sentence[i].replace("\u200d"," ")
                df.sentence[i]=df.sentence[i].replace("\u200e"," ")
                df.sentence[i]=df.sentence[i].replace("…"," ")
            else:
                df.sentence[i] = pred["text"]+'।'
                df.sentence[i]=df.sentence[i].replace("<>"," ")
                df.sentence[i]=df.sentence[i].replace("  "," ")
                df.sentence[i]=df.sentence[i].replace("<unk>"," ")
                df.sentence[i]=df.sentence[i].replace("<s>"," ")
                df.sentence[i]=df.sentence[i].replace("</s>"," ")
                df.sentence[i]=df.sentence[i].replace("f"," ")
                df.sentence[i]=df.sentence[i].replace("F"," ")
                df.sentence[i]=df.sentence[i].replace("\u200c"," ")
                df.sentence[i]=df.sentence[i].replace("\u200d"," ")
                df.sentence[i]=df.sentence[i].replace("\u200e"," ")
                df.sentence[i]=df.sentence[i].replace("…"," ")

        except:
            df.sentence[i] = pred["text"]+'।'
            df.sentence[i]=df.sentence[i].replace("<>"," ")
            df.sentence[i]=df.sentence[i].replace("  "," ")
            df.sentence[i]=df.sentence[i].replace("<unk>"," ")
            df.sentence[i]=df.sentence[i].replace("<s>"," ")
            df.sentence[i]=df.sentence[i].replace("</s>"," ")
            df.sentence[i]=df.sentence[i].replace("f"," ")
            df.sentence[i]=df.sentence[i].replace("F"," ")
            df.sentence[i]=df.sentence[i].replace("\u200c"," ")
            df.sentence[i]=df.sentence[i].replace("\u200d"," ")
            df.sentence[i]=df.sentence[i].replace("\u200e"," ")
            df.sentence[i]=df.sentence[i].replace("…"," ")
        
    df.sentence=df.sentence.parallel_apply(lambda x:normalize(x)) #unicode normalizer
    return df


directory test

In [ ]:
TEST_DIRECTORY = '../input/test-wav-files-dl-sprint/test_files_wav/'
submission=directory_infer(TEST_DIRECTORY)
submission.to_csv('./submission.csv',index = False)
submission.head(50)

In [ ]:
Sample_wav_file ='../input/test-wav-files-dl-sprint/test_files_wav/common_voice_bn_30993879.wav'
prediction = infer(Sample_wav_file)
prediction